# indexing the rhyme

In [1]:
import os
import sys
import logging
PRJ_BASE ='/mnt/d/personal_working/bavandangvanba'
base_path = PRJ_BASE
sys.path.append(base_path)
from core.utils import (
    vn_grammar_handler,
    db_handler,
    init_logger)
init_logger()

/mnt/d/personal_working/bavandangvanba/


In [2]:
import pandas as pd
n_gram_phrases_song = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_songs_data.pkl'))
n_gram_phrases_truyen = pd.read_pickle(os.path.join(PRJ_BASE,'tmp/dictionary_from_truyen_data.pkl'))
n_gram_phrases = n_gram_phrases_truyen
for key,count in n_gram_phrases_song.items():
    if key not in n_gram_phrases:
        n_gram_phrases[key] = count*10
    else:
        n_gram_phrases[key] += count*10
        
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")
pho_vocab = tokenizer.get_vocab()
for key,count in pho_vocab.items():
    real_count = 55000 // (count + 1)
    if real_count<1:
        continue
    if key not in n_gram_phrases:
        n_gram_phrases[key] = real_count
    else:
        n_gram_phrases[key] += real_count

/home/giangtran/miniconda3/envs/streamlit_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
len(n_gram_phrases)

11355412

In [4]:
wrong_data = []
van_indexing = {} 
for idx,(word, count) in enumerate(n_gram_phrases.items()):
    cleaned_text = vn_grammar_handler.bo_dau_va_chuyen_van(word)
    matches_01 = vn_grammar_handler.lay_van_cua_tu_hoac_doan(cleaned_text)
    matches_02 = vn_grammar_handler.clean_phu_am(cleaned_text)
    if matches_01!=matches_02:
        wrong_data.append(['something wrong', word, matches_01, matches_02])
    if matches_01 not in van_indexing:
        van_indexing[matches_01] = []
    van_indexing[matches_01].append((word,count))
for van in van_indexing:
    van_indexing[van].sort(key=lambda x:x[1],reverse=True)
del van_indexing[""]


In [5]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example.db'))

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create a table if it doesn't exist
cursor.execute('''CREATE TABLE IF NOT EXISTS van_indexing
                  (id TEXT PRIMARY KEY, words TEXT)''')

In [6]:
for idx,(van, text) in enumerate(van_indexing.items()):
    text_list = "\n".join(x[0] for x in text)
   # Add data to the table
    data = ('1', 'Hello, world!')
    cursor.execute('INSERT INTO van_indexing (id, words) VALUES (?, ?)', (van,text_list))
    if idx % 10000 == 0:
        logging.info(f'  - processed {idx} van')

2023-11-11 11:53:31 INFO       - processed 0 van
2023-11-11 11:53:32 INFO       - processed 10000 van
2023-11-11 11:53:32 INFO       - processed 20000 van
2023-11-11 11:53:32 INFO       - processed 30000 van
2023-11-11 11:53:32 INFO       - processed 40000 van
2023-11-11 11:53:33 INFO       - processed 50000 van
2023-11-11 11:53:33 INFO       - processed 60000 van
2023-11-11 11:53:33 INFO       - processed 70000 van
2023-11-11 11:53:33 INFO       - processed 80000 van
2023-11-11 11:53:34 INFO       - processed 90000 van
2023-11-11 11:53:34 INFO       - processed 100000 van
2023-11-11 11:53:35 INFO       - processed 110000 van
2023-11-11 11:53:36 INFO       - processed 120000 van
2023-11-11 11:53:36 INFO       - processed 130000 van
2023-11-11 11:53:37 INFO       - processed 140000 van
2023-11-11 11:53:38 INFO       - processed 150000 van
2023-11-11 11:53:39 INFO       - processed 160000 van
2023-11-11 11:53:40 INFO       - processed 170000 van
2023-11-11 11:53:41 INFO       - processed

In [7]:
conn.commit()

# Close the connection
conn.close()

In [16]:
import subprocess

source_file_path = "/mnt/d/personal_working/bavandangvanba/app_metadata/example.db"
destination_file_path = "giangtran@ec2-52-74-160-204.ap-southeast-1.compute.amazonaws.com:/home/giangtran/working/bavandangvanba/app_metadata/example.db"
key_file_path = "/home/giangtran/.ssh/giang_key.pem"

subprocess.run(["scp", "-i", key_file_path, source_file_path, destination_file_path])

CompletedProcess(args=['scp', '-i', '/home/giangtran/.ssh/giang_key.pem', '/mnt/d/personal_working/bavandangvanba/app_metadata/example.db', 'giangtran@ec2-52-74-160-204.ap-southeast-1.compute.amazonaws.com:/home/giangtran/working/bavandangvanba/app_metadata/example.db'], returncode=0)

# test old db

In [1]:
import os
import sys
import logging
PRJ_BASE ='/mnt/d/personal_working/bavandangvanba'
base_path = PRJ_BASE
sys.path.append(base_path)
from core.utils import (
    vn_grammar_handler,
    db_handler,
    init_logger)
init_logger()

/mnt/d/personal_working/bavandangvanba/


In [10]:
import sqlite3
# conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example_bk_12M.db'))
conn = sqlite3.connect(os.path.join(PRJ_BASE,'app_metadata/example.db'))
cursor = conn.cursor()


In [11]:
van = "ang e"
cursor.execute(f"SELECT words FROM van_indexing WHERE id = '{van}'")
rows = cursor.fetchall()
rows

[('dáng vẻ\nnàng sẽ\ndáng mẹ\nđang nghe\nhàng tre\nđáng lẽ\nnàng nghe\ncàng sẽ\nđang hé\nhàng me\nvàng nhẹ\ncàng nghe\nnàng khẽ\nsáng sẽ\nđang khẽ\nnàng ghé\nnàng nhẹ\nđang vẽ\nsáng nghe\nnàng thẻ\ntháng mẹ\nđang nhẹ\nchàng sẽ\nmạng che\nnàng che\nmang vẻ\ntháng sẽ\nnàng đè\nsang hè\nchang the\nđang xé\nmang mẹ\nsang ghé\nsang mẹ\nđang se\nsang sẽ\nnàng vẽ\nvàng khẽ\nnàng vẻ\nsáng che\ntang de\nngang sẽ\nlàng xe\ntháng đẻ\nnàng hé\ncàng che\nnàng xé\nđang ghé\ncàng trẻ\nvang khẽ\nđáng nghe\nvang nhẹ\nvàng ghé\nsang ve\nđang bé\nnàng mẹ\nnàng xe\nhàng sẽ\nmang xe\nnàng té\ncàng xe\nvàng vẻ\nmang nhẹ\ndáng vẽ\nngang che\nlàng e\nsang kẻ\nvàng se\nQuang Le\nsang nè\nđang ngh e\nhàng e\ncàng bé\nđang phe\nđang đè\nsáng nhẹ\nlạng lẽ\ncàng bẻ\ngang de\nsáng vẽ\nmang khẽ\nđáng nhẽ\nhạng vé\nvàng xe\nvàng nghe\nsáng mẹ\nmang me\ncàng ghẹ\nnàng nè\ndang the\nmang thẻ\nquang sẽ\nnàng đẻ\ncàng nhẹ\ntràng sẽ\nnàng bẻ\nquang nhẹ\nquang né\nnàng nhe\nmang sẽ\nquang che\nhàng nghe\ngiang nghe\nnàng k

In [12]:
vn_grammar_handler.bo_dau_va_chuyen_van("làng quê")

'lang quê'

In [13]:
vn_grammar_handler.clean_phu_am(vn_grammar_handler.bo_dau_va_chuyen_van("làng quê"))

'ang ê'

In [14]:
vn_grammar_handler.lay_van_cua_tu_hoac_doan(vn_grammar_handler.bo_dau_va_chuyen_van("làng quê"))

'ang ê'

In [15]:
vn_grammar_handler.pattern_remove_phu_am

re.compile(r'((^|\s)ngh(?!\s))|((^|\s)ch(?!\s))|((^|\s)gh(?!\s))|((^|\s)gi(?!\s))|((^|\s)kh(?!\s))|((^|\s)ng(?!\s))|((^|\s)nh(?!\s))|((^|\s)ph(?!\s))|((^|\s)qu(?!\s))|((^|\s)th(?!\s))|((^|\s)tr(?!\s))|((^|\s)b(?!\s))|((^|\s)c(?!\s))|((^|\s)d(?!\s))|((^|\s)g(?!\s))|((^|\s)h(?!\s))|((^|\s)k(?!\s))|((^|\s)l(?!\s))|((^|\s)m(?!\s))|((^|\s)n(?!\s))|((^|\s)p(?!\s))|((^|\s)q(?!\s))|((^|\s)r(?!\s))|((^|\s)s(?!\s))|((^|\s)t(?!\s))|((^|\s)v(?!\s))|((^|\s)x(?!\s))|((^|\s)đ(?!\s))',
           re.UNICODE)

In [8]:
import re

pattern = r"(?:(?<=\s))ngh(?!\s)"
text = "This is an example nghwith ngh at the beginning ngh\tand ngh ngh\nin different situations."

result = re.sub(pattern, "", text)
print(result)

This is an example with ngh at the beginning ngh	and ngh ngh
in different situations.
